# Notebook that generates encoded data to use for ML VPD


## Imports and Configs
Make sure to change the datasetDirectory variable to the path of the PreprocessedImages folder

In [1]:
!pip install pillow
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
from PIL import Image
from PIL import ImageFilter
from scipy import ndimage
import numpy as np
import os
import sys
import math
import matplotlib.pyplot as plt

datasetDirectory = "../Data/"
directory = os.fsencode(datasetDirectory)

NUM_STAIRS = 2

## Setup directories

In [3]:
print(len(os.listdir(directory)))

2299


In [4]:
depthImagePaths = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("_Range.png"): 
        depthImagePaths.append(filename)
    else:
        continue
            
if not os.path.isdir(datasetDirectory + '/Dataset/SCDEncoded/Training'):
    os.makedirs(datasetDirectory + '/Dataset/SCDEncoded/Training')
if not os.path.isdir(datasetDirectory + '/Dataset/BetaMap/Training'):
    os.makedirs(datasetDirectory + '/Dataset/BetaMap/Training')

if not os.path.isdir(datasetDirectory + '/Dataset/SCDEncoded/Testing'):
    os.makedirs(datasetDirectory + '/Dataset/SCDEncoded/Testing')
if not os.path.isdir(datasetDirectory + '/Dataset/BetaMap/Testing'):
    os.makedirs(datasetDirectory + '/Dataset/BetaMap/Testing')

## Define SCD Encode Function

In [5]:
def scdEncode(file):
    # Get image depth data
    gaussian_kernel = [[1 / 16.0, 2 / 16.0, 1 / 16.0],
                       [2 / 16.0, 4 / 16.0, 2 / 16.0],
                       [1 / 16.0, 2 / 16.0, 1 / 16.0]]
    
    depthImage = Image.open(datasetDirectory + '/' + depthImagePaths[file])
    depthArray = np.asarray(depthImage)
    depthArray = depthArray.astype(float)
    depthArray = ndimage.convolve(depthArray, gaussian_kernel, mode='constant', cval=0.0)
    
    # Get information about depth
    rows = depthArray.shape[0]
    cols = depthArray.shape[1]
    zmax = np.max(depthArray)
    zmin = np.min(depthArray)
    zrng = zmax - zmin
    
    # Determine encoding parameters
    nstr = NUM_STAIRS
    P = zrng / nstr
    Beta = P / 2
    
    # SCD encoding
    I1 = np.zeros((rows,cols))
    I2 = np.zeros((rows,cols))
    I3 = np.zeros((rows,cols))
    depthMask = np.zeros((rows,cols))
    I1temp = np.zeros((rows,cols))
    I2temp = np.zeros((rows,cols))
    
    
    I1 = 0.5 + 0.5 * np.cos(2 * np.pi * depthArray / P)
    
    I1[np.where(depthArray <= 0)] = 0
    depthMask[np.where(depthArray > 0)] = 1
        
    I1Image = Image.fromarray((I1*255).astype(np.uint8))
    
    return (depthArray, I1, I2, I3, I1Image, zrng, Beta, depthMask, nstr)

## Define Image Saving Function

In [6]:
def saveImages(file, orgZ, I1, I2, I3, I1Image, zrng, Beta, depthMask, nstr):
    # Save out SCD encoded image
    SCDArray = np.dstack((I1*255, I2*255, I3*255)) # Combine three channels into one array
    SCDImage = Image.fromarray((SCDArray).astype(np.uint8)) # Grayscale
    SCDImageFilename = depthImagePaths[file].split('.') # Formatting filename
    if file < np.ceil(0.2 * len(depthImagePaths)):
        SCDImage.save(datasetDirectory + '/Dataset/SCDEncoded/Testing/' + SCDImageFilename[0] + '_SCD_Encoded.png')
    else:
        SCDImage.save(datasetDirectory + '/Dataset/SCDEncoded/Training/' + SCDImageFilename[0] + '_SCD_Encoded.png')

    # Calculate beta map
    CorrectBetaMap = np.floor(orgZ / Beta)
    CorrectBetaMap = CorrectBetaMap + depthMask # This should keep the background as zero while making all other values <1, 2, 3...>
    threshold_indices = (CorrectBetaMap == (2 * nstr + 1))
    CorrectBetaMap[threshold_indices] = 2 * nstr   # This avoids the 'nipple' that occurs at the highest point in the range


    # Save out beta map
    BetaImage = Image.fromarray(CorrectBetaMap.astype(np.uint8), 'L') # Grayscale
    BetaImageFilename = depthImagePaths[file].split('.') # Formatting filename
    if file < np.ceil(0.2 * len(depthImagePaths)):
        BetaImage.save(datasetDirectory + '/Dataset/BetaMap/Testing/' + BetaImageFilename[0] + '_SCD_Encoded_Beta_Map.png')
    else:
        BetaImage.save(datasetDirectory + '/Dataset/BetaMap/Training/' + BetaImageFilename[0] + '_SCD_Encoded_Beta_Map.png')


## Use SCD Encode function on images

In [7]:
for file in range(len(depthImagePaths)):
#for file in range(0, 1):
    # Progress indicator
    if file < np.ceil(0.2 * len(depthImagePaths)):
        sys.stdout.write('\rGenerating Testing Set: %d / %d' % (file + 1, np.ceil(0.2 * len(depthImagePaths))))
        sys.stdout.flush()
    else:
        if file == np.ceil(0.2 * len(depthImagePaths)):
            sys.stdout.write('\n')
            sys.stdout.flush()
    
        sys.stdout.write('\rGenerating Training Set: %d / %d' % (file + 1 - np.ceil(0.2 * len(depthImagePaths)), len(depthImagePaths) - np.ceil(0.2 * len(depthImagePaths))))
        sys.stdout.flush()
        
    orgZ, I1, I2, I3, I1Image, zrng, Beta, depthMask, nstr = scdEncode(file)
    saveImages(file, orgZ, I1, I2, I3, I1Image, zrng, Beta, depthMask, nstr)
    
print('\nDataset Generated!')

Generating Testing Set: 230 / 230
Generating Training Set: 919 / 919
Dataset Generated!
